In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif
from scipy.stats import ttest_ind, mannwhitneyu

# ───────────────────────────────────────────────
# 1. Load Control and PD data
# ───────────────────────────────────────────────
con_df = pd.read_csv(r"E:\USA_PD_2024\Analysis\ppr6\COP\ML\Feature_Selection_2\Data\Controlled1.csv")
pd_df  = pd.read_csv(r"E:\USA_PD_2024\Analysis\ppr6\COP\ML\Feature_Selection_2\Data\PD1.csv")

# Strip column names (important)
con_df.columns = con_df.columns.str.strip()
pd_df.columns  = pd_df.columns.str.strip()

# Add label column
con_df["label"] = 0
pd_df["label"] = 1

# Combine into one DataFrame
df = pd.concat([con_df, pd_df], ignore_index=True)

# ───────────────────────────────────────────────
# 2. Preprocess: Drop meta-columns
# ───────────────────────────────────────────────
drop_cols = ["label", "File"]  # Demographics already removed
X = df.drop(columns=drop_cols, errors="ignore")
y = df["label"].values  # 0 = Control, 1 = PD

# ───────────────────────────────────────────────
# 3. Run per-feature statistical tests
# ───────────────────────────────────────────────
results = []

for col in X.columns:
    ctrl_vals = X[y == 0][col]
    pd_vals   = X[y == 1][col]

    # Welch's t-test (unequal variance)
    t_stat, t_pval = ttest_ind(ctrl_vals, pd_vals, equal_var=False)

    # Mann–Whitney U test
    u_stat, u_pval = mannwhitneyu(ctrl_vals, pd_vals, alternative="two-sided")

    results.append({
        "Feature": col,
        "Control_Mean": np.mean(ctrl_vals),
        "PD_Mean": np.mean(pd_vals),
        "T-test_p": t_pval,
        "U-test_p": u_pval,
        "Mean_Diff": abs(np.mean(ctrl_vals) - np.mean(pd_vals))
    })

# Convert to DataFrame
res_df = pd.DataFrame(results)

# ───────────────────────────────────────────────
# 4. Add ANOVA F-test and p-values
# ───────────────────────────────────────────────
f_vals, f_pvals = f_classif(X, y)
res_df["ANOVA_F"] = f_vals
res_df["ANOVA_p"] = f_pvals

# ───────────────────────────────────────────────
# 5. Add significance flags and sort
# ───────────────────────────────────────────────
res_df["p<0.05"] = res_df["ANOVA_p"] < 0.05
res_df["p<0.01"] = res_df["ANOVA_p"] < 0.01

res_df = res_df.sort_values("ANOVA_p")

# ───────────────────────────────────────────────
# 6. Show and Save Results
# ───────────────────────────────────────────────
# Show top 20 most significant features
print("\n=== Top 20 Most Significant Features (by ANOVA p-value) ===")
print(res_df[["Feature", "Control_Mean", "PD_Mean", "Mean_Diff", "ANOVA_F", "ANOVA_p", "T-test_p", "U-test_p"]].head(20))

# Save all results
res_df.to_csv("significant_features_summary.csv", index=False)
print("\n✅ Results saved to 'significant_features_summary.csv'")



=== Top 20 Most Significant Features (by ANOVA p-value) ===
                                               Feature  Control_Mean  \
125  Feature_asym_energy_content_below_05_Power_Spe...      0.155235   
26             Feature_avg_fractal_dimension_ML_AND_AP      2.773514   
45   Feature_avg_power_frequency_95_Power_Spectrum_...      2.737618   
121  Feature_asym_centroid_frequency_Power_Spectrum...      0.035792   
44   Feature_avg_power_frequency_95_Power_Spectrum_...      2.758763   
59   Feature_avg_frequency_quotient_Power_Spectrum_...      0.117110   
117  Feature_asym_power_frequency_95_Power_Spectrum...      0.048024   
49   Feature_avg_centroid_frequency_Power_Spectrum_...      1.278092   
132     Feature_asym_short_time_diffusion_Diffusion_ML      0.061559   
131  Feature_asym_frequency_quotient_Power_Spectrum...      0.140286   
48   Feature_avg_centroid_frequency_Power_Spectrum_...      1.304404   
52   Feature_avg_energy_content_below_05_Power_Spec...      0.196923   
89 

In [7]:
"""
Binary PD vs Control classification using manually selected CoP + demographic features
──────────────────────────────────────────────────────────────────────────────────────
Features: 15 CoP-derived + Height, Weight, Sex (if available)
Modeling: SVM, RF, LR, k-NN, GNB
Validation: 5-fold GroupKFold using SubjectID
"""

# ──────────────────────────────────────────────────────────────
# 1. Imports
# ──────────────────────────────────────────────────────────────
import re
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.base import clone

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# ──────────────────────────────────────────────────────────────
# 2. Load and concatenate data
# ──────────────────────────────────────────────────────────────
con_df = pd.read_csv(r"E:\USA_PD_2024\Analysis\ppr6\COP\ML\Feature_Selection_2\Data\Controlled1.csv")
pd_df  = pd.read_csv(r"E:\USA_PD_2024\Analysis\ppr6\COP\ML\Feature_Selection_2\Data\PD1.csv")

con_df["label"] = 0
pd_df ["label"] = 1
df = pd.concat([con_df, pd_df], ignore_index=True)

# ──────────────────────────────────────────────────────────────
# 3. Clean column names
# ──────────────────────────────────────────────────────────────
df.columns = df.columns.str.strip().str.replace(" ", "_")

# ──────────────────────────────────────────────────────────────
# 4. Extract Subject ID from filename
# ──────────────────────────────────────────────────────────────
def get_id(fname: str) -> str:
    base = Path(fname).stem
    m = re.search(r"_[a-zA-Z]*([0-9]+)", base)
    return m.group(1) if m else base

df["SubjectID"] = df["File"].apply(get_id)

# ──────────────────────────────────────────────────────────────
# 5. Manually selected features
# ──────────────────────────────────────────────────────────────
manual_features = [
    "Feature_asym_energy_content_below_05_Power_Spectrum_Density_AP",
    "Feature_avg_fractal_dimension_ML_AND_AP",
    "Feature_avg_power_frequency_95_Power_Spectrum_Density_AP",
    "Feature_asym_centroid_frequency_Power_Spectrum_Density_AP",
    "Feature_avg_power_frequency_95_Power_Spectrum_Density_ML",
    "Feature_avg_frequency_quotient_Power_Spectrum_Density_AP",
    "Feature_asym_power_frequency_95_Power_Spectrum_Density_AP",
    "Feature_avg_centroid_frequency_Power_Spectrum_Density_AP",
    "Feature_asym_short_time_diffusion_Diffusion_ML",
    "Feature_asym_frequency_quotient_Power_Spectrum_Density_AP",
    "Feature_avg_centroid_frequency_Power_Spectrum_Density_ML",
    "Feature_avg_energy_content_below_05_Power_Spectrum_Density_ML",
    "Feature_asym_confidence_ellipse_area_ML_AND_AP",
    "Feature_avg_mean_frequency_ML_AND_AP",
    "Feature_avg_short_time_scaling_Diffusion_AP",
    "Feature_avg_LFS_ML_AND_AP",
    "Feature_asym_rms_AP",
    "Feature_asym_long_time_diffusion_Diffusion_AP",
    "Feature_asym_critical_displacement_Diffusion_AP",
    "Feature_asym_LFS_ML_AND_AP"
]

# Check availability
available_features = [f for f in manual_features if f in df.columns]
missing = set(manual_features) - set(available_features)
if missing:
    print(f"⚠️ Warning: Missing features skipped: {missing}")
else:
    print("✅ All selected features are present.")

# ──────────────────────────────────────────────────────────────
# 6. Prepare feature matrix X
# ──────────────────────────────────────────────────────────────
if "Sex" in df.columns:
    X = df[available_features + ["Sex"]].copy()
    X = pd.get_dummies(X, columns=["Sex"], drop_first=True)
else:
    X = df[available_features].copy()
    print("⚠️ 'Sex' column not found. Proceeding without it.")

y = df["label"].values
groups = df["SubjectID"].values

# ──────────────────────────────────────────────────────────────
# 7. Cross-validation setup
# ──────────────────────────────────────────────────────────────
cv = GroupKFold(n_splits=5)

# ──────────────────────────────────────────────────────────────
# 8. Models and parameter grids
# ──────────────────────────────────────────────────────────────
models = [
    ("SVM (RBF)", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", probability=True, random_state=42))
    ]), {"clf__C": [0.1, 1, 10], "clf__gamma": ["scale", 0.1]}),

    ("Random Forest", Pipeline([
        ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))
    ]), {"clf__n_estimators": [100], "clf__max_depth": [None]}),

    ("Logistic Regression", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=500, random_state=42))
    ]), {"clf__C": [1]}),

    ("k-NN", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier())
    ]), {"clf__n_neighbors": [5, 7]}),

    ("Gaussian NB", Pipeline([
        ("clf", GaussianNB())
    ]), {})
]

# ──────────────────────────────────────────────────────────────
# 9. Train, tune and evaluate
# ──────────────────────────────────────────────────────────────
for name, pipe, param_grid in models:
    print(f"\n🔹 {name} Model 🔹")
    
    # Grid Search CV
    gs = GridSearchCV(pipe, param_grid, scoring="f1", cv=cv, n_jobs=-1)
    gs.fit(X, y, groups=groups)
    best_est = gs.best_estimator_

    # Evaluate with group-wise CV
    accs, precs, recs, f1s, aucs = [], [], [], [], []
    for tr, te in cv.split(X, y, groups):
        best_est.fit(X.iloc[tr], y[tr])
        preds = best_est.predict(X.iloc[te])
        probs = (best_est.predict_proba(X.iloc[te])[:, 1]
                 if hasattr(best_est, "predict_proba")
                 else best_est.decision_function(X.iloc[te]))

        accs.append(accuracy_score(y[te], preds))
        precs.append(precision_score(y[te], preds, zero_division=0))
        recs.append(recall_score(y[te], preds, zero_division=0))
        f1s.append(f1_score(y[te], preds, zero_division=0))
        aucs.append(roc_auc_score(y[te], probs))

    # Print metrics
    print(f"Best Params: {gs.best_params_}")
    print(f"Accuracy : {np.mean(accs):.3f} ± {np.std(accs):.3f}")
    print(f"Precision: {np.mean(precs):.3f} ± {np.std(precs):.3f}")
    print(f"Recall   : {np.mean(recs):.3f} ± {np.std(recs):.3f}")
    print(f"F1-score : {np.mean(f1s):.3f} ± {np.std(f1s):.3f}")
    print(f"ROC-AUC  : {np.mean(aucs):.3f} ± {np.std(aucs):.3f}")


✅ All selected features are present.
⚠️ 'Sex' column not found. Proceeding without it.

🔹 SVM (RBF) Model 🔹
Best Params: {'clf__C': 1, 'clf__gamma': 0.1}
Accuracy : 0.713 ± 0.098
Precision: 0.716 ± 0.149
Recall   : 0.700 ± 0.132
F1-score : 0.692 ± 0.090
ROC-AUC  : 0.798 ± 0.091

🔹 Random Forest Model 🔹
Best Params: {'clf__max_depth': None, 'clf__n_estimators': 100}
Accuracy : 0.748 ± 0.093
Precision: 0.751 ± 0.151
Recall   : 0.760 ± 0.109
F1-score : 0.740 ± 0.076
ROC-AUC  : 0.823 ± 0.108

🔹 Logistic Regression Model 🔹
Best Params: {'clf__C': 1}
Accuracy : 0.791 ± 0.051
Precision: 0.782 ± 0.126
Recall   : 0.795 ± 0.077
F1-score : 0.778 ± 0.047
ROC-AUC  : 0.856 ± 0.092

🔹 k-NN Model 🔹
Best Params: {'clf__n_neighbors': 5}
Accuracy : 0.696 ± 0.087
Precision: 0.683 ± 0.097
Recall   : 0.665 ± 0.138
F1-score : 0.665 ± 0.093
ROC-AUC  : 0.739 ± 0.077

🔹 Gaussian NB Model 🔹
Best Params: {}
Accuracy : 0.748 ± 0.080
Precision: 0.756 ± 0.128
Recall   : 0.720 ± 0.084
F1-score : 0.727 ± 0.068
ROC-AUC